In [2]:
# ------------------------------------
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.
# ------------------------------------

"""
DESCRIPTION:
    This sample demonstrates how to use agent operations with the 
    Azure AI Search tool from the Azure Agents service using a synchronous client.
    To learn how to set up an Azure AI Search resource,
    visit https://learn.microsoft.com/azure/search/search-get-started-portal

USAGE:
    Before running the sample:
    Set these environment variables with your own values:
    1) AZURE_AISTUDIO_PROJECT_CONN_STRING - The project connection string, as found in the overview page of your
       Azure AI Foundry project.
    2) AZURE_OPENAI_GPT4o_DEPLOYMENT_NAME - The deployment name of the AI model, as found under the "Name" column in 
       the "Models + endpoints" tab in your Azure AI Foundry project.
    3) AZURE_AI_SEARCH_CONNECTION_ID - The connection ID of the Azure AI Search resource, as found in the "Connections" tab
"""
import os
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import CodeInterpreterTool, AzureAISearchTool, ConnectionType
from azure.identity import DefaultAzureCredential
from typing import Any
from pathlib import Path
from dotenv import load_dotenv

# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# At the moment, it should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<ProjectName>"
# HostName can be found by navigating to your discovery_url and removing the leading "https://" and trailing "/discovery"
# To find your discovery_url, run the CLI command: az ml workspace show -n {project_name} --resource-group {resource_group_name} --query discovery_url
# Project Connection example: eastus.api.azureml.ms;12345678-abcd-1234-9fc6-62780b3d3e05;my-resource-group;my-project-name
# Customer needs to login to Azure subscription via Azure CLI and set the environment variables

load_dotenv()
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_GPT4o_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_GPT4o_DEPLOYMENT_NAME")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_AISTUDIO_PROJECT_CONN_STRING = os.getenv("AZURE_AISTUDIO_PROJECT_CONN_STRING")

In [3]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=AZURE_AISTUDIO_PROJECT_CONN_STRING
)


conn_list = project_client.connections.list()
conn_id = "cognsearchopenaiworkshop"
for conn in conn_list:
    if conn.connection_type == ConnectionType.AZURE_AI_SEARCH:
        conn_id = conn.id
        break

print(conn_id)

# Initialize agent AI search tool and add the search index connection id
ai_search = AzureAISearchTool(index_connection_id=conn_id, index_name="books")

# Create agent with AI search tool and process assistant run
with project_client:
    agent = project_client.agents.create_agent(
        model=AZURE_OPENAI_GPT4o_DEPLOYMENT_NAME,
        name="my-assistant",
        instructions="You are a helpful assistant",
        tools=ai_search.definitions,
        tool_resources=ai_search.resources,
        headers={"x-ms-enable-preview": "true"},
    )
    # [END create_agent_with_azure_ai_search_tool]
    print(f"Created agent, ID: {agent.id}")

    # Create thread for communication
    thread = project_client.agents.create_thread()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="Why does the coffin prepared for Queequeg become Ishmael's life buoy once the Pequod sinks?",
    )
    print(f"Created message, ID: {message.id}")

    # Create and process agent run in thread with tools
    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Delete the assistant when done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

    # Fetch and log all messages
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

/subscriptions/ec967cb5-f2b0-43c2-9ba2-4a2eb94bbacc/resourceGroups/aistudio-pf-rag/providers/Microsoft.MachineLearningServices/workspaces/pf-rag/connections/cognsearchopenaiworkshop
Created agent, ID: asst_LmfkXIp3VGqmoi4cWqYaH3PZ
Created thread, ID: thread_KCSbGOlaBhksCzzTgD1oil7g
Created message, ID: msg_KSg6N4id9HehW6e9PZq3bsGk
Run finished with status: completed
Deleted agent
Messages: {'object': 'list', 'data': [{'id': 'msg_BbBieTvJ3h0nn3x6EIy8nBQe', 'object': 'thread.message', 'created_at': 1739109383, 'assistant_id': 'asst_LmfkXIp3VGqmoi4cWqYaH3PZ', 'thread_id': 'thread_KCSbGOlaBhksCzzTgD1oil7g', 'run_id': 'run_nWKWUDITirFZeiimdKodpRql', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'In Herman Melville\'s novel "Moby-Dick," Queequeg, a harpooner on the Pequod, has a coffin made for himself when he believes he is going to die. However, he eventually recovers and no longer needs the coffin. Later in the story, after the Pequod sinks, the coffin is repurposed 

In [4]:
# print(messages["data"][0]["content"][0]["text"]["value"])

# Fetch and log all messages in chronological order
messages_data = messages["data"]

# Sort messages by creation time (ascending)
sorted_messages = sorted(messages_data, key=lambda x: x["created_at"])

print("\n--- Thread Messages (sorted) ---")
for msg in sorted_messages:
    role = msg["role"].upper()
    # Each 'content' is a list; get the first text block if present
    content_blocks = msg.get("content", [])
    text_value = ""
    if content_blocks and content_blocks[0]["type"] == "text":
        text_value = content_blocks[0]["text"]["value"]
    print(f"{role}: {text_value}")


--- Thread Messages (sorted) ---
USER: Why does the coffin prepared for Queequeg become Ishmael's life buoy once the Pequod sinks?
ASSISTANT: In Herman Melville's novel "Moby-Dick," Queequeg, a harpooner on the Pequod, has a coffin made for himself when he believes he is going to die. However, he eventually recovers and no longer needs the coffin. Later in the story, after the Pequod sinks, the coffin is repurposed as a life buoy that saves Ishmael, the story's narrator and sole survivor of the shipwreck.

The transformation of Queequeg's coffin into a life buoy carries significant symbolic meaning. It represents the themes of life and death, survival, and the unpredictability of fate. The coffin, an object traditionally associated with death, becomes an instrument of life for Ishmael, emphasizing the cyclical nature of existence and the idea that out of death can come renewal and salvation.
